In [1]:
import os
import pandas as pd
import numpy as np

import tensorflow as tf

In [2]:
train = pd.read_csv('data/train_features.csv')
train_labels = pd.read_csv('data/train_labels.csv')
test = pd.read_csv('data/test_features.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [3]:
# id랑 time label을 없앰
X=tf.reshape(np.array(train.iloc[:,2:]),[-1, 600, 6])
X.shape

TensorShape([3125, 600, 6])

In [4]:
# label을 one-hot encoding (61개의 unique 한 label 존재)
y = tf.keras.utils.to_categorical(train_labels['label']) 
y.shape

(3125, 61)

In [8]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Activation, LSTM, BatchNormalization
from keras import optimizers

def deep_lstm():
    model = Sequential()
    model.add(LSTM(32, input_shape = (600,6), return_sequences = True))
    model.add(BatchNormalization())
    model.add(LSTM(64, return_sequences = True))
    model.add(BatchNormalization())
    model.add(LSTM(128, return_sequences = False))
    model.add(BatchNormalization())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(61, activation='softmax'))
    
    #adam = optimizers.adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
    return model

In [9]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [10]:
model1 = deep_lstm()
model1.fit(X,y, epochs=100, batch_size=32, verbose = 1,  validation_split=0.2, callbacks = [es])

Epoch 1/100
79/79 [==============================] - 5666s 72s/step - loss: 3.2532 - accuracy: 0.3654 - val_loss: 3.3475 - val_accuracy: 0.4976
Epoch 2/100
79/79 [==============================] - 88s 1s/step - loss: 2.3739 - accuracy: 0.4782 - val_loss: 2.7941 - val_accuracy: 0.5040
Epoch 3/100
79/79 [==============================] - 84s 1s/step - loss: 2.0401 - accuracy: 0.5119 - val_loss: 2.4628 - val_accuracy: 0.4960
Epoch 4/100
79/79 [==============================] - 87s 1s/step - loss: 1.7535 - accuracy: 0.5378 - val_loss: 2.3604 - val_accuracy: 0.5072
Epoch 5/100
79/79 [==============================] - 91s 1s/step - loss: 1.5510 - accuracy: 0.5791 - val_loss: 2.4272 - val_accuracy: 0.5136
Epoch 6/100
79/79 [==============================] - 96s 1s/step - loss: 1.3898 - accuracy: 0.5993 - val_loss: 2.4881 - val_accuracy: 0.4928
Epoch 7/100
79/79 [==============================] - 110s 1s/step - loss: 1.1685 - accuracy: 0.6700 - val_loss: 2.6304 - val_accuracy: 0.4784
Epoch 8/1

In [11]:
test_X=tf.reshape(np.array(test.iloc[:,2:]),[-1, 600, 6])
test_X.shape
prediction=model1.predict(test_X)
submission.iloc[:,1:]=prediction

In [12]:
submission.to_csv('deepLSTM_es_bn_submission.csv', index=False)

In [13]:
stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
model1 = deep_lstm()
model1.fit(X,y, epochs=300, batch_size=32, verbose = 1,  validation_split=0.2, callbacks = [stop])

Epoch 1/300
79/79 [==============================] - 177s 2s/step - loss: 3.1734 - accuracy: 0.3855 - val_loss: 3.3197 - val_accuracy: 0.4976
Epoch 2/300
79/79 [==============================] - 145s 2s/step - loss: 2.3133 - accuracy: 0.5068 - val_loss: 2.8342 - val_accuracy: 0.5136
Epoch 3/300
79/79 [==============================] - 139s 2s/step - loss: 2.0842 - accuracy: 0.4974 - val_loss: 2.5142 - val_accuracy: 0.4912
Epoch 4/300
79/79 [==============================] - 144s 2s/step - loss: 1.8468 - accuracy: 0.5413 - val_loss: 2.4035 - val_accuracy: 0.5040
Epoch 5/300
79/79 [==============================] - 113s 1s/step - loss: 1.6644 - accuracy: 0.5573 - val_loss: 2.4427 - val_accuracy: 0.4928
Epoch 6/300
79/79 [==============================] - 96s 1s/step - loss: 1.5107 - accuracy: 0.5731 - val_loss: 2.4748 - val_accuracy: 0.4832
Epoch 7/300
79/79 [==============================] - 99s 1s/step - loss: 1.3136 - accuracy: 0.6084 - val_loss: 2.6781 - val_accuracy: 0.4656
Epoch 8/

KeyboardInterrupt: 

In [ ]:
test_X=tf.reshape(np.array(test.iloc[:,2:]),[-1, 600, 6])
test_X.shape
prediction=model1.predict(test_X)
submission.iloc[:,1:]=prediction
submission.to_csv('output/deepLSTM_es_bn_v2_submission.csv', index=False)